# EV Charging Infrastructure Demand Analysis

This notebook analyzes demand patterns and generates location recommendations based on:
1. Previously collected charging station and potential location data
2. Population density from Statistics Canada
3. Coverage and accessibility metrics

## Key Objectives:
- Analyze population coverage of existing infrastructure
- Identify high-demand areas
- Score potential locations for new stations
- Generate recommendations for optimization model

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium import plugins
from pathlib import Path
from datetime import datetime

from src.data.api_client import APIClient
from src.data.constants import *
from src.data.utils import *
from src.visualization import map_viz

# Initialize API client (only needed for population data)
client = APIClient()

## 1. Load and Prepare Data
Load our previously processed location data and fetch population density information.

In [2]:
# Get latest processed location data
analyzed_locations_file, al_timestamp = get_latest_csv(DATA_PATHS['analyzed_locations'])
print(f"Loading previously analyzed location data from {al_timestamp}")
print(f"File: {analyzed_locations_file}")
analyzed_locations_df = pd.read_csv(analyzed_locations_file)
analyzed_locations_df.head()

Loading previously analyzed location data from 2024-11-12 22:57:44
File: /u1/a9dutta/co370/kw-ev-charging-optimization/data/processed/analyzed_locations/analyzed_locations_2024-11-12_22-57-44.csv


,name,latitude,longitude,num_chargers,charger_type,operator,address,city,postal_code,usage_cost,data_source,location_type,area,geometry_type,min_distance_to_station
0,Metropolitan Towers,43.460806,-80.512639,1.0,Level 2,SWTCH - Charge Everywhere,57 Union St E,Waterloo,N2J 1C1,Unknown,OpenChargeMap,charging_station,NaN,NaN,0.0
1,WATERLOO REGION,43.463730,-80.519593,1.0,Level 2,ChargePoint,100 Regina Street South,Waterloo,N2J 4A8,Unknown,OpenChargeMap,charging_station,NaN,NaN,0.0
2,William Street,43.463168,-80.519858,1.0,Level 2,flo,29 William Street East,Waterloo,N2L 1J4,Unknown,OpenChargeMap,charging_station,NaN,NaN,0.0
3,Christian Horizon - North Lot,43.467422,-80.518764,1.0,Level 2,flo,26 Peppler St,Waterloo,N2J 3C4,Unknown,OpenChargeMap,charging_station,NaN,NaN,0.0
4,Bridgeport Plaza,43.470145,-80.515211,1.0,Level 2,SWTCH - Charge Everywhere,70 Bridgeport Rd E,Waterloo,N2J 2J9,Unknown,OpenChargeMap,charging_station,NaN,NaN,0.0


In [3]:
# Initialize API client and fetch population data
client = APIClient()
population_df = client.fetch_population_density()

# Save population data
pd_timestamp = grab_time()
population_file = DATA_PATHS['population_density'] / f'population_density_{pd_timestamp}.csv'
population_df.to_csv(population_file, index=False)
print(f"Population data saved to: {population_file}")

Fetching population density data from OpenStreetMap...

📊 Data Collection Summary
Generated: November 12, 2024 at 23:00:45

📍 Data Sources and Versions:
• OpenStreetMap (OSM):
  - OSMnx Version: 1.9.4
  - PyProj Version: 3.7.0

🏘️ Processing cities...

📍 Processing Kitchener...
   Fetching residential buildings...


   ✓ Found 9,102 buildings
   Processing building data...


100%|██████████████████████████████| 9102/9102 [00:01<00:00, 9100.00it/s]



   Fetching amenities...
   ✓ Found 91 amenities
   Processing amenity data...


100%|██████████████████████████████| 91/91 [00:00<00:00, 9300.24it/s]



📍 Processing Waterloo...
   Fetching residential buildings...
   ✓ Found 7,896 buildings
   Processing building data...


100%|██████████████████████████████| 7896/7896 [00:00<00:00, 10368.69it/s]



   Fetching amenities...
   ✓ Found 43 amenities
   Processing amenity data...


100%|██████████████████████████████| 43/43 [00:00<00:00, 8294.86it/s]



Features Found:
  - Total Features: 17,132
    ∟ 16,998 residential buildings
    ∟ 134 public amenities

📊 Regional Summary by City:

Kitchener:
  Residential:
    • Buildings: 9,102
    • Est. Population: 383,473
    • Total Area: 1.90 km²
  Amenities:
    • Locations: 91
    • Daily Population: 235,500
    • Total Area: 7.06 km²

Waterloo:
  Residential:
    • Buildings: 7,896
    • Est. Population: 245,756
    • Total Area: 1.54 km²
  Amenities:
    • Locations: 43
    • Daily Population: 143,500
    • Total Area: 5.66 km²
Population data saved to: /u1/a9dutta/co370/kw-ev-charging-optimization/data/raw/population_density/population_density_2024-11-12_23-00-49.csv


In [4]:
# Updated Data Summary
print("\nData Summary:")
print(f"Total locations: {len(analyzed_locations_df)}")
print(f"- Existing stations: {len(analyzed_locations_df[analyzed_locations_df['data_source'] == 'OpenChargeMap'])}")
print(f"- Potential locations: {len(analyzed_locations_df[analyzed_locations_df['data_source'] == 'OpenStreetMap'])}")
print(f"Population density points: {len(population_df)}")

print("\nData Timestamps:")
print(f"Population Data: {pd_timestamp}")
print(f"Analyzed Locations Data: {al_timestamp}")


Data Summary:
Total locations: 3683
- Existing stations: 132
- Potential locations: 3551
Population density points: 17132

Data Timestamps:
Population Data: 2024-11-12_23-00-49
Analyzed Locations Data: 2024-11-12 22:57:44


In [5]:
def create_population_map(population_df, title="Population Density"):
    """Create a map with population heatmap."""
    m = map_viz.create_kw_map()
    
    # Create locations list and convert weights to Python list
    locations = [[row['latitude'], row['longitude']] 
                for _, row in population_df.iterrows()]
    weights = population_df['population'].values.tolist()  # Convert numpy array to list
    
    folium.plugins.HeatMap(
        locations,
        weights=weights,
        radius=15,
        blur=20,
        max_zoom=13,
        name=title
    ).add_to(m)
    
    # Add layer control
    folium.LayerControl().add_to(m)
    
    return m

# Create summary statistics
print("Population Statistics:")
total_pop = population_df['population'].sum()
print(f"Total Population: {total_pop:,.0f}")

# Create separate maps for different population types
print("\nGenerating population density maps...")

# 1. Overall population
print("\nOverall Population Distribution:")
m_all = create_population_map(population_df, "All Population")
display(m_all)

# 2. Residential population
residential_mask = population_df['location_type'] == 'residential'
residential_df = population_df[residential_mask]
residential_pop = residential_df['population'].sum()
print(f"\nResidential Population: {residential_pop:,.0f} ({residential_pop/total_pop:.1%} of total)")
m_residential = create_population_map(residential_df, "Residential Population")
display(m_residential)

# 3. Amenity population
amenity_mask = population_df['location_type'] == 'amenity'
amenity_df = population_df[amenity_mask]
amenity_pop = amenity_df['population'].sum()
print(f"\nAmenity Daily Population: {amenity_pop:,.0f} ({amenity_pop/total_pop:.1%} of total)")

# Show amenity breakdown
print("\nAmenity Breakdown:")
amenity_stats = (amenity_df.groupby('amenity_type')
                .agg({
                    'population': ['count', 'sum', 'mean']
                })
                .round(2))
print(amenity_stats)

m_amenity = create_population_map(amenity_df, "Amenity Population")
display(m_amenity)

Population Statistics:
Total Population: 1,008,229

Generating population density maps...

Overall Population Distribution:



Residential Population: 629,229 (62.4% of total)



Amenity Daily Population: 379,000 (37.6% of total)

Amenity Breakdown:
             population                    
                  count       sum      mean
amenity_type                               
college               4   75500.0  18875.00
school              123  123000.0   1000.00
university            7  180500.0  25785.71


In [6]:
# Save processed population data
print("\nSaving processed population data...")
output_file = DATA_PATHS['population_analysis'] / f'population_analysis_{grab_time()}.csv'
population_df.to_csv(output_file, index=False)
print(f"Saved to: {output_file}")


Saving processed population data...
Saved to: /u1/a9dutta/co370/kw-ev-charging-optimization/data/processed/population_analysis/population_analysis_2024-11-12_23-00-51.csv


## 2. Current Coverage Analysis
Let's analyze how well the existing charging stations cover the population:

In [7]:
def calculate_coverage_metrics(points_df, population_df, radius_km=2):
    """Calculate population coverage within radius of points."""
    covered_population = 0
    total_population = population_df['population'].sum()
    
    # Create point geometries
    points_gdf = gpd.GeoDataFrame(
        points_df,
        geometry=gpd.points_from_xy(points_df.longitude, points_df.latitude),
        crs="EPSG:4326"
    )
    pop_gdf = gpd.GeoDataFrame(
        population_df,
        geometry=gpd.points_from_xy(population_df.longitude, population_df.latitude),
        crs="EPSG:4326"
    )
    
    # Project to UTM for accurate distance calculation
    points_gdf = points_gdf.to_crs("EPSG:32617")
    pop_gdf = pop_gdf.to_crs("EPSG:32617")
    
    # Calculate coverage
    for _, pop_point in pop_gdf.iterrows():
        distances = points_gdf.geometry.distance(pop_point.geometry)
        if (distances <= radius_km * 1000).any():  # Convert km to meters
            covered_population += pop_point['population']
    
    return {
        'covered_population': covered_population,
        'coverage_percentage': (covered_population / total_population) * 100,
        'total_population': total_population
    }

In [8]:
# Calculate current coverage
residential_mask = population_df['location_type'] == 'residential'
residential_df = population_df[residential_mask]

charging_mask = analyzed_locations_df['data_source'] == 'OpenChargeMap'
charging_df = analyzed_locations_df[charging_mask]

coverage = calculate_coverage_metrics(charging_df, residential_df)

# Display coverage statistics
print("Current Coverage Analysis:")
print("-" * 50)
print(f"Total Population: {coverage['total_population']:,.0f}")
print(f"Population Covered: {coverage['covered_population']:,.0f}")
print(f"Coverage Percentage: {coverage['coverage_percentage']:.1f}%")

Current Coverage Analysis:
--------------------------------------------------
Total Population: 629,229
Population Covered: 607,901
Coverage Percentage: 96.6%


In [17]:
# Check the unique values in the charger_type column
print(charging_df['charger_type'].unique())

# Visualize coverage
plt.figure(figsize=(12, 8))
m = map_viz.create_kw_map()

# Add population density heatmap
locations = [[row['latitude'], row['longitude']] 
            for _, row in residential_df.iterrows()]
weights = residential_df['population'].tolist()  # Convert to list

folium.plugins.HeatMap(
    locations,
    weights=weights,
    radius=15,
    blur=20,
    max_zoom=13
).add_to(m)

# Add existing charging stations
for _, station in charging_df.iterrows():
    folium.CircleMarker(
        location=[station['latitude'], station['longitude']],
        radius=8,
        color='red',
        fill=True,
        popup=f"{station['name']}<br>{str(station['charger_type'])}"  # Ensure charger_type is a string
    ).add_to(m)

m

['Level 2' 'Level 3']


<Figure size 1200x800 with 0 Axes>